In [1]:
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_name = 'CSCI544-Project'
    folder_path = f'/content/drive/MyDrive/{folder_name}'
    os.chdir(folder_path)
    print("Current working directory is: " + os.getcwd())
    BASE_EMBEDDINGS_PATH = "./t5p_small_embeddings/"
except:
    BASE_EMBEDDINGS_PATH = "../t5p_small_embeddings/"
finally:
    print("Current working directory is: " + os.getcwd())
    print(os.listdir())

Current working directory is: /Users/swapnildhanwal/Documents/USC/CSCI 544/Project/repeat/src
['embeddings.ipynb', 'training.ipynb', 'model.py', 'utils.py', 'data.ipynb', 'replay.ipynb']


In [2]:
import torch

from model import FNNTopology, Model
from utils import load_tensors

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
X_train, y_train = load_tensors(BASE_EMBEDDINGS_PATH + "train/train_0.jsonl")
model = Model(FNNTopology("FNN with averaged vectors", X_train.shape[1]))
print("Initial model training...")
model.train(X_train, y_train)
scores = []
for i in range(1, 5):
    train_file_path = BASE_EMBEDDINGS_PATH + "train/train_%d.jsonl" % i
    test_file_path = BASE_EMBEDDINGS_PATH + "train/train_%d.jsonl" % i
    X_train, y_train = load_tensors(train_file_path)
    X_test, y_test = load_tensors(test_file_path)
    print("Continual learning iteration %d..." % i)
    model.train(X_train, y_train, 100, 100)
    score = model.evaluate(X_test, y_test)
    scores.append(score)
    print(score)
print(scores)

Initial model training...


TypeError: train() missing 2 required positional arguments: 'epochs' and 'batch_size'